# Pandas (part 2)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt #Vẽ chart

In [ ]:
# Load data
data_path = 'D:\\codegym\\data\\'
data_name = 'output_OnlineRetail.csv'
df = pd.read_csv(data_path + data_name, encoding= 'unicode_escape')

In [ ]:
# Head
df.head()

In [ ]:
df.shape

## 2. Thống kê mô tả

- Doanh thu thấp nhất trên 1 hóa đơn

In [ ]:
df["Revenue"].min()

- Kiểm tra xem kết quả âm là do đâu

In [ ]:
# Lọc ra những dòng có revenue < 0
df.loc[df["Revenue"] < 0, :].head()

- Nguyên nhân: quantity có thể âm 
    - Quantity < 0: công ty nhập hàng vào
    - Quantity > 0: công ty bán hàng ra

- Do vậy cần lọc ra những hóa đơn liên quan đến BÁN HÀNG

In [ ]:
df_sell = df.loc[df["Quantity"] > 0, :].copy()
df_sell.shape

In [ ]:
df_sell["Quantity"].min()

- Tính lại giá trị hóa đơn nhỏ nhất

In [ ]:
df_sell["Revenue"].min()

In [ ]:
df_sell["UnitPrice"].min()

- Chúng ta cần làm sạch data bằng cách loại bỏ UnitPrice < 0

In [ ]:
# Đọc lại data gốc:
df = pd.read_csv(data_path + data_name, encoding= 'unicode_escape')

# Loại bỏ UnitPrice < 0 và lưu vào 1 dataframe khác
df_true_price = df.loc[df["UnitPrice"] >= 0, :].copy()

- Tính giá trị hóa đơn cao nhất

In [ ]:
df_true_price["Revenue"].max()

- Tính doanh thu trung bình trên mỗi hóa đơn

In [ ]:
df_true_price["Revenue"].mean()

- Tính median doanh thu trên mỗi hóa đơn

In [ ]:
df_true_price["Revenue"].median()

## 3. Group by and aggregation

- Chỉ sử dụng group by cho những cột có unique values nhỏ
- Luôn luôn đặt aggregration ngay sau group by

### 3.1. Examples with group by (1 column)

In [ ]:
# Mean quantity by country
df_true_price.groupby("Country")["Quantity"].mean().nlargest(10)

In [ ]:
# Median quantity by country
df_true_price.groupby("Country")["Quantity"].median().nlargest(10)

Note: results are `Series`

In [ ]:
# Save the aggregation to result
result = df_true_price.groupby("Country")["Quantity"].median().nlargest(10)

In [ ]:
# Verify the type
type(result)

In [ ]:
# Access Male index
result.loc["Netherlands"]

In [ ]:
# Or just like this
result["Netherlands"]

- More than 1 summary statistics (dùng `.agg`)

In [ ]:
# Mean and median bill by sex
df_true_price.groupby("Country")["Quantity"].agg(["min", "mean", "median", "max",  "std"])

In [ ]:
# A shortcut
df_true_price.groupby("Country")["Quantity"].describe()

In [ ]:
df_true_price["Quantity"].describe()['75%']

- Reset index để tiếp tục tính toán (nếu có)

In [ ]:
df_true_price.groupby("Country")["Quantity"].describe().reset_index()

### 3.2. Examples with group by (multiple columns)

In [ ]:
# Mean Quantity by Country and GoodType
df_true_price.groupby(["Country", "GoodType"])["Quantity"].mean()

- Note: Kết quả phía trên là một multi-index Series (2 levels)

In [ ]:
# Lưu vào result
result = df_true_price.groupby(["Country", "GoodType"])["Quantity"].mean()

In [ ]:
# Mean quantity for the goods that Cheap and in United Kingdom
result[("United Kingdom", "Cheap")]

In [ ]:
# Mean quantity for the goods that Luxury and in Singapore
result[("Singapore", "Luxury")]

- Ta cũng có thể reset index để được kết quả là 1 data frame

In [ ]:
result.reset_index()

- Tương tự như group by 1 column, ta cũng có thể dùng `.describe()`

In [ ]:
df_true_price.groupby(["Country", "GoodType"])["Quantity"].describe()

- Reset index

In [ ]:
df_true_price.groupby(["Country", "GoodType"])["Quantity"].describe().reset_index()

## 4. Một số thao tác khác với columns

### 4.1 Đổi tên cột

In [ ]:
# Preview
df_true_price.head(1)

In [ ]:
# Tạo df2 gồm 3 cột đầu
df2 = df_true_price.iloc[:, :3]
df2.head(1)

- Đổi tên cột `StockCode` thành `stock_code` và `InvoiceNo` thành `invoice_no` (cách 1)
- Cách này phù hợp cho sửa tên một số ít cột

In [ ]:
df2.rename(columns={"StockCode": "stock_code", "InvoiceNo": "invoice_no"}, inplace=True)
df2.head(2)

- Đổi tên tất cả 3 cột thành `stock_code`, `invoice_no`, và `desc`
- Cách này phù hợp khi đổi tên hết tất cả các cột (lưu ý giữ đúng thứ tự)

In [ ]:
df2.columns = ["stock_code", "invoice_no", "desc"]
df2.head(1)

## 5. Ép kiểu dữ liệu cột

In [ ]:
# Preview
df_true_price.head(1)

In [ ]:
# Dtypes
df_true_price.dtypes

- Ép cột `Quantity` thành string

In [ ]:
df_true_price["Quantity"] = df_true_price["Quantity"].astype(str)
df_true_price.dtypes

- Ép ngược lại `Quantity` thành int

In [ ]:
df_true_price["Quantity"] = df_true_price["Quantity"].astype(int)
df_true_price.dtypes

- Ép cột `InvoiceDate` thành datetime

In [ ]:
df_true_price["InvoiceDate"] = pd.to_datetime(df_true_price["InvoiceDate"])
df_true_price.dtypes